<a href="https://www.kaggle.com/code/lakshmikanth1010/task-on-machine-learning-1?scriptVersionId=170834984" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**Develop a predictive model using a machine learning algorithm and perform cross-validation to assess its performance.**

In [1]:
#LET IMPORT AS LIBRARIES
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
#Load the dataset
data=pd.read_csv("/kaggle/input/machine-learning-task/Dataset - Data Science - Task 2.csv")
data

,Make,Model,Year,Mileage,Fuel Type,Gearbox Type,Price
0,Toyota,Camry,2010,120000.0,Petrol,Manual,8000.0
1,Ford,Focus,2008,100000.0,Diesel,Manual,5000.0
2,BMW,X5,2015,80000.0,Petrol,Automatic,25000.0
3,Tesla,Model S,2017,60000.0,Electric,Automatic,45000.0
4,Ford,Focus,2005,150000.0,Diesel,Manual,3000.0
5,NaN,NaN,2012,90000.0,Petrol,Manual,7000.0
6,BMW,NaN,2013,75000.0,Diesel,Automatic,20000.0
7,Toyota,Camry,2010,NaN,Petrol,Manual,NaN
8,Ford,Focus,2010,130000.0,Diesel,NaN,6000.0
9,Tesla,Model 3,2019,30000.0,Electric,Automatic,35000.0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Make          9 non-null      object 
 1   Model         8 non-null      object 
 2   Year          10 non-null     int64  
 3   Mileage       9 non-null      float64
 4   Fuel Type     10 non-null     object 
 5   Gearbox Type  9 non-null      object 
 6   Price         9 non-null      float64
dtypes: float64(2), int64(1), object(4)
memory usage: 688.0+ bytes


# DATA CLEANING

In [4]:
# Check for missing values
data.isnull().sum()

Make            1
Model           2
Year            0
Mileage         1
Fuel Type       0
Gearbox Type    1
Price           1
dtype: int64

In [5]:
# Fill missing values for categorical variables with the mode
for column in ['Make', 'Model', 'Gearbox Type']:
    data[column].fillna(data[column].mode()[0], inplace=True) #warning for fillna function

/tmp/ipykernel_18/2984163354.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].mode()[0], inplace=True) #warning for fillna function


In [6]:
# Fill missing values for numerical variables with the median
for column in ['Mileage', 'Price']:
    data[column].fillna(data[column].median(), inplace=True)    #warning for fillna function

/tmp/ipykernel_18/1236372612.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].median(), inplace=True)    #warning for fillna function
/tmp/ipykernel_18/1236372612.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col

In [7]:
# Check again for missing values to confirm
data.isnull().sum()

Make            0
Model           0
Year            0
Mileage         0
Fuel Type       0
Gearbox Type    0
Price           0
dtype: int64

In [8]:
# Selecting categorical columns for one-hot encoding
categorical_columns = ['Make', 'Model', 'Fuel Type', 'Gearbox Type']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[categorical_columns])

In [9]:
# Using get_feature_names for compatibility
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

# Drop original categorical columns and concatenate the encoded DataFrame
df_encoded = pd.concat([data.drop(categorical_columns, axis=1), encoded_df], axis=1)

In [10]:
# Display the first few rows of the new dataframe
df_encoded.head()

,Year,Mileage,Price,Make_BMW,Make_Ford,Make_Tesla,Make_Toyota,Model_Camry,Model_Focus,Model_Model 3,Model_Model S,Model_X5,Fuel Type_Diesel,Fuel Type_Electric,Fuel Type_Petrol,Gearbox Type_Automatic,Gearbox Type_Manual
0,2010,120000.0,8000.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,2008,100000.0,5000.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2015,80000.0,25000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,2017,60000.0,45000.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,2005,150000.0,3000.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [11]:
# Assuming 'Price' is the target variable and the rest are features
X = data.drop('Price', axis=1)
y = data['Price']

In [12]:
# Step 4: Encode categorical variables using one-hot encoding
X = pd.get_dummies(X, columns=['Make', 'Model', 'Fuel Type', 'Gearbox Type'], drop_first=True)

In [13]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Training set size:', X_train.shape[0])
print('Testing set size:', X_test.shape[0])

Training set size: 8
Testing set size: 2


In [14]:
# Initialize the model
model = LinearRegression()

In [15]:
# Train the model on the training set
model.fit(X_train, y_train)

LinearRegression()

In [16]:
#Evaluate the model using cross-validation
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [17]:
# Calculate mean squared error for training and testing sets
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

print("Mean Squared Error (Training):", mse_train)
print("Mean Squared Error (Testing):", mse_test)

Mean Squared Error (Training): 3.4450524030726903e-17
Mean Squared Error (Testing): 7043485.684782974


In [18]:
# Initializing the StandardScaler
scaler = StandardScaler()

In [19]:
# Fitting the scaler to the features and transforming them
X_scaled = scaler.fit_transform(X)

In [20]:
# Displaying the first few rows of the scaled features to verify
pd.DataFrame(X_scaled, columns=X.columns).head()

,Year,Mileage,Make_Ford,Make_Tesla,Make_Toyota,Model_Focus,Model_Model 3,Model_Model S,Model_X5,Fuel Type_Electric,Fuel Type_Petrol,Gearbox Type_Manual
0,-0.473670,0.830573,-0.816497,-0.5,2.0,-1.0,-0.333333,-0.333333,-0.333333,-0.5,1.224745,0.816497
1,-0.972269,0.226520,1.224745,-0.5,-0.5,1.0,-0.333333,-0.333333,-0.333333,-0.5,-0.816497,0.816497
2,0.772829,-0.377533,-0.816497,-0.5,-0.5,-1.0,-0.333333,-0.333333,3.000000,-0.5,1.224745,-1.224745
3,1.271429,-0.981586,-0.816497,2.0,-0.5,-1.0,-0.333333,3.000000,-0.333333,2.0,-0.816497,-1.224745
4,-1.720169,1.736653,1.224745,-0.5,-0.5,1.0,-0.333333,-0.333333,-0.333333,-0.5,-0.816497,0.816497


## cross-validataion Result

**To analyze the cross-validation results and identify potential areas of improvement for the model, we would typically look at metrics such as the mean and standard deviation of the model's performance across different folds. These metrics can indicate whether the model is overfitting (if the model performs significantly better on the training data compared to the validation data) or underfitting (if the model performs poorly on both training and validation data).**
**Based on the general approach to model evaluation and improvement, here are two strategies that could be suggested:**

**Parameter Tuning: Adjusting the hyperparameters of the model can help in finding a better balance between bias and variance, potentially reducing overfitting or underfitting. For a linear regression model, this might involve regularization techniques such as Lasso (L1 regularization) or Ridge (L2 regularization) to penalize large coefficients that can lead to overfitting.**

**Feature Selection: Analyzing the importance of different features and removing those that do not contribute significantly to the model's predictive power can improve model performance. This can also help in reducing the complexity of the model, making it less prone to overfitting. Techniques such as backward elimination, forward selection, or using model-based methods like feature importance scores from tree-based models can be employed for feature selection.**

## Summary of Insights from Model Training and Validation:

* **The process of model training and validation is crucial for understanding the performance of the model under different conditions and identifying areas for improvement.**
* **Cross-validation helps in assessing the model's ability to generalize to unseen data by using multiple splits of the data.**
* **Potential limitations faced during this process could include computational constraints when dealing with large datasets or a high number of features, which can make extensive parameter tuning and cross-validation computationally expensive.**
* **Addressing these limitations could involve using more efficient algorithms, feature selection to reduce the dimensionality of the data, or employing parallel computing techniques to speed up the computation.**
* **Future work could focus on exploring more sophisticated models or ensemble methods that might offer better performance, as well as experimenting with different feature engineering techniques to extract more informative features from the data.**
*      **These strategies and insights provide a roadmap for refining the model and achieving better performance in predicting car prices.**